<a href="https://colab.research.google.com/github/Hussien-Sayed/Solar-PAYGo-Payment-Prediction/blob/main/Solar_PAYGo_Payment_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
!pip install catboost

In [ ]:
# Import libraries
import ast
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor


from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.max_columns', None)

## Exploring data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load files
train = pd.read_csv("/content/drive/MyDrive/zindi/solar_zindi_comp/Train.csv")
metadata = pd.read_csv("/content/drive/MyDrive/zindi/solar_zindi_comp/metadata.csv")
test = pd.read_csv("/content/drive/MyDrive/zindi/solar_zindi_comp/Test.csv")
ss = pd.read_csv("/content/drive/MyDrive/zindi/solar_zindi_comp/SampleSubmission.csv")

<ipython-input-11-e660a9cc0db1>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("/content/drive/MyDrive/zindi/solar_zindi_comp/metadata.csv")


In [ ]:
#remove leaked feature
metadata = metadata[metadata.columns.difference(['LastPaymentDate'])]

In [ ]:
# check data shapes
train.shape, test.shape, metadata.shape, ss.shape

((28007, 9), (9336, 3), (37343, 19), (56016, 2))

In [ ]:
train.shape[0]+test.shape[0], metadata.shape[0]

(37343, 37343)

In [ ]:
ss.shape[0], test.shape[0]*6

(56016, 56016)

In [ ]:
# Preview train
train.head(10)

,ID,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6
0,ID_MR53LEX,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[3600.0, 750.0, 350.0, 65.0, 95.0, 135.0, 85.0...",880.0,930.0,495.0,715.0,220.0,385.0
1,ID_3D7NQUH,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[2940.0, 970.0, 380.0, 880.0, 385.0, 440.0, 11...",660.0,935.0,935.0,825.0,770.0,935.0
2,ID_0IWQNPI,"['02-2020', '03-2020', '04-2020', '05-2020', '...","[2850.0, 1500.0, 1350.0, 610.0, 200.0, 250.0]",700.0,1350.0,1550.0,1400.0,1450.0,1200.0
3,ID_IY8SYB9,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2200.0, 1420.0, 1180.0, 900.0, 1400.0, 780.0,...",580.0,480.0,800.0,1260.0,1650.0,530.0
4,ID_9XHL7VZ,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2640.0, 910.0, 480.0, 280.0, 200.0, 180.0, 33...",40.0,440.0,460.0,360.0,80.0,330.0
5,ID_8XNJ47Q,"['07-2017', '08-2017', '09-2017', '10-2017', '...","[2000.0, 1100.0, 988.0, 1040.0, 1084.0, 1160.0...",1160.0,1130.0,1140.0,1090.0,1240.0,490.0
6,ID_KI932AD,"['08-2017', '02-2018', '04-2018', '05-2018', '...","[2280.0, 40.0, 40.0, 40.0, 80.0, 40.0, 55.0, 5...",40.0,40.0,40.0,40.0,80.0,80.0
7,ID_3TMLZ41,"['03-2018', '04-2018', '05-2018', '06-2018', '...","[4485.0, 1805.0, 2035.0, 1595.0, 930.0, 1815.0...",2090.0,1320.0,2090.0,1210.0,1660.0,454.0
8,ID_GPL8VO8,"['02-2018', '03-2018', '04-2018', '05-2018', '...","[3270.0, 100.0, 1650.0, 1715.0, 1000.0, 1265.0...",1740.0,750.0,1150.0,750.0,1150.0,1576.0
9,ID_KMK2R2J,"['01-2020', '02-2020', '03-2020', '04-2020', '...","[2800.0, 850.0, 1180.0, 1500.0, 720.0, 1090.0]",680.0,1040.0,860.0,1000.0,680.0,400.0


In [ ]:
metadata.head(10)

,AccessoryRate,Age,DaysOnDeposit,Deposit,ExpectedTermDate,FirstPaymentDate,ID,MainApplicantGender,Occupation,PaymentMethod,RatePerUnit,Region,RegistrationDate,SupplierName,Term,TotalContractValue,Town,UpsellDate,rateTypeEntity
0,0.0,41.0,7,2000,2016-12-08 00:00:00,2015-12-10 09:52:35,ID_K00S4N4,Male,Other,FINANCED,35,Mount Kenya Region,2015-12-10 00:00:00,d_light,364,14740.0,Embu,NaN,DAILY
1,0.0,33.0,7,2000,2016-12-07 00:00:00,2015-12-09 13:14:03,ID_6L67PAA,Male,Other,FINANCED,35,Coast Region,2015-12-09 00:00:00,d_light,364,14740.0,Kilifi,NaN,DAILY
2,35.0,48.0,7,2000,2017-01-13 00:00:00,2015-12-18 06:22:34,ID_102CV85,Female,Business,FINANCED,35,Nairobi Region,2015-12-18 00:00:00,d_light,392,29480.0,Makueni,2018-03-29 10:14:58,DAILY
3,0.0,43.0,7,2000,2016-11-23 00:00:00,2015-11-25 13:25:57,ID_HXBJFHB,Female,Teacher,FINANCED,35,NaN,2015-11-25 00:00:00,d_light,364,14740.0,UNKNOWN,NaN,DAILY
4,0.0,56.0,7,2000,2016-11-30 00:00:00,2015-12-05 10:34:32,ID_3K9VZ5J,Female,Other,FINANCED,35,Mount Kenya Region,2015-12-02 00:00:00,d_light,364,14740.0,Kirinyaga,NaN,DAILY
5,0.0,NaN,7,2000,2016-12-12 00:00:00,2015-12-14 14:20:26,ID_VJ80SX2,Female,Business,FINANCED,35,Mount Kenya Region,2015-12-14 00:00:00,d_light,364,14740.0,Embu,NaN,DAILY
6,0.0,56.0,7,2000,2016-12-08 00:00:00,2015-12-10 15:15:16,ID_6YU5YRB,Male,Other,FINANCED,35,Mount Kenya Region,2015-12-10 00:00:00,d_light,364,14740.0,Meru,NaN,DAILY
7,0.0,41.0,7,2000,2016-12-09 00:00:00,2015-12-11 10:34:10,ID_FK5A5E9,Female,Other,FINANCED,35,Mount Kenya Region,2015-12-11 00:00:00,d_light,364,14740.0,Embu,NaN,DAILY
8,0.0,32.0,7,2000,2016-12-04 00:00:00,2015-12-06 16:41:21,ID_7OU9HLK,Male,Business,FINANCED,35,Mount Kenya Region,2015-12-06 00:00:00,d_light,364,14740.0,Embu,NaN,DAILY
9,0.0,42.0,7,2000,2016-12-03 00:00:00,2015-12-04 18:11:33,ID_WVWTPGK,Female,Other,FINANCED,35,Mount Kenya Region,2015-12-05 00:00:00,d_light,364,14740.0,Kirinyaga,NaN,DAILY


In [ ]:
test.head(10)

,ID,TransactionDates,PaymentsHistory
0,ID_6L67PAA,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[4000.0, 1050.0, 1050.0, 1050.0, 1050.0, 400.0]"
1,ID_VJ80SX2,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[3000.0, 850.0, 750.0, 1500.0, 650.0, 1250.0, ..."
2,ID_7OU9HLK,"['12-2015', '01-2016', '03-2016', '05-2016', '...","[2400.0, 300.0, 500.0, 450.0, 675.0, 700.0, 87..."
3,ID_WVWTPGK,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[4700.0, 1200.0, 950.0, 1200.0, 900.0, 1110.0,..."
4,ID_04DSDQS,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[4800.0, 750.0, 995.0, 995.0, 1300.0, 750.0, 1..."
5,ID_4BNXJX1,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[6600.0, 1060.0, 1065.0, 1025.0, 915.0, 820.0,..."
6,ID_6YZEOBD,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[2710.0, 980.0, 1050.0, 1120.0, 1015.0, 1050.0..."
7,ID_7AKUMTZ,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[4580.0, 1000.0, 975.0, 1010.0, 1025.0, 1078.0..."
8,ID_22PA8JG,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[2500.0, 1100.0, 1000.0, 1050.0, 1100.0, 1120...."
9,ID_TM3XAXK,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[2400.0, 35.0, 620.0, 715.0, 455.0, 340.0, 105..."


In [ ]:
ss.head(10)

,ID,Target
0,ID_6L67PAA x m1,0.0
1,ID_6L67PAA x m2,0.0
2,ID_6L67PAA x m3,0.0
3,ID_6L67PAA x m4,0.0
4,ID_6L67PAA x m5,0.0
5,ID_6L67PAA x m6,0.0
6,ID_VJ80SX2 x m1,0.0
7,ID_VJ80SX2 x m2,0.0
8,ID_VJ80SX2 x m3,0.0
9,ID_VJ80SX2 x m4,0.0


In [ ]:
m_train = pd.merge(left=metadata, right=train, on="ID")
m_test = pd.merge(left=metadata, right=test, on="ID")

In [ ]:
m_train.shape,m_test.shape

((28007, 27), (9336, 21))

In [ ]:
m_train.head(10)

,AccessoryRate,Age,DaysOnDeposit,Deposit,ExpectedTermDate,FirstPaymentDate,ID,MainApplicantGender,Occupation,PaymentMethod,RatePerUnit,Region,RegistrationDate,SupplierName,Term,TotalContractValue,Town,UpsellDate,rateTypeEntity,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6
0,0.0,41.0,7,2000,2016-12-08 00:00:00,2015-12-10 09:52:35,ID_K00S4N4,Male,Other,FINANCED,35,Mount Kenya Region,2015-12-10 00:00:00,d_light,364,14740.0,Embu,NaN,DAILY,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[3050.0, 1050.0, 910.0, 1050.0, 1050.0]",1225.0,1050.0,1190.0,525.0,1750.0,1890.0
1,35.0,48.0,7,2000,2017-01-13 00:00:00,2015-12-18 06:22:34,ID_102CV85,Female,Business,FINANCED,35,Nairobi Region,2015-12-18 00:00:00,d_light,392,29480.0,Makueni,2018-03-29 10:14:58,DAILY,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[4245.0, 980.0, 735.0, 735.0, 1470.0, 735.0, 9...",980.0,980.0,1225.0,980.0,935.0,355.0
2,0.0,43.0,7,2000,2016-11-23 00:00:00,2015-11-25 13:25:57,ID_HXBJFHB,Female,Teacher,FINANCED,35,NaN,2015-11-25 00:00:00,d_light,364,14740.0,UNKNOWN,NaN,DAILY,"['11-2015', '12-2015', '01-2016', '02-2016', '...","[2245.0, 980.0, 980.0, 1225.0, 980.0, 980.0, 9...",250.0,1000.0,250.0,500.0,560.0,1150.0
3,0.0,56.0,7,2000,2016-11-30 00:00:00,2015-12-05 10:34:32,ID_3K9VZ5J,Female,Other,FINANCED,35,Mount Kenya Region,2015-12-02 00:00:00,d_light,364,14740.0,Kirinyaga,NaN,DAILY,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[2750.0, 1000.0, 750.0, 1000.0, 955.0, 880.0, ...",630.0,805.0,700.0,855.0,245.0,775.0
4,0.0,56.0,7,2000,2016-12-08 00:00:00,2015-12-10 15:15:16,ID_6YU5YRB,Male,Other,FINANCED,35,Mount Kenya Region,2015-12-10 00:00:00,d_light,364,14740.0,Meru,NaN,DAILY,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[3010.0, 665.0, 1035.0, 985.0, 1020.0, 1120.0,...",1050.0,985.0,1085.0,980.0,1770.0,90.0
5,0.0,41.0,7,2000,2016-12-09 00:00:00,2015-12-11 10:34:10,ID_FK5A5E9,Female,Other,FINANCED,35,Mount Kenya Region,2015-12-11 00:00:00,d_light,364,14740.0,Embu,NaN,DAILY,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[4750.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000....",855.0,675.0,1015.0,950.0,920.0,325.0
6,0.0,43.0,7,2000,2016-12-07 00:00:00,2015-12-09 09:23:37,ID_EL1XKC7,Male,Other,FINANCED,35,Nairobi Region,2015-12-09 00:00:00,d_light,364,14740.0,Kitui,NaN,DAILY,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[2760.0, 850.0, 1250.0, 1050.0, 1000.0, 1400.0...",1100.0,1050.0,900.0,1200.0,800.0,780.0
7,0.0,56.0,7,2000,2016-12-07 00:00:00,2015-12-09 10:50:24,ID_P0KAW56,Female,Business,FINANCED,35,Mount Kenya Region,2015-12-09 00:00:00,d_light,364,14740.0,Embu,NaN,DAILY,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[2750.0, 1140.0, 1000.0, 994.0, 1310.0, 940.0,...",1005.0,1098.0,970.0,805.0,1025.0,745.0
8,0.0,36.0,7,2000,2016-12-08 00:00:00,2015-12-10 08:17:23,ID_C0PKZEV,Female,Other,FINANCED,35,Mount Kenya Region,2015-12-10 00:00:00,d_light,364,14740.0,Nyeri,NaN,DAILY,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[2770.0, 970.0, 950.0, 1035.0, 990.0, 985.0, 6...",35.0,50.0,40.0,35.0,35.0,70.0
9,0.0,28.0,7,2000,2016-12-08 00:00:00,2015-12-10 08:49:37,ID_JJ4DQI7,Male,Other,FINANCED,35,Mount Kenya Region,2015-12-10 00:00:00,d_light,364,14740.0,Embu,NaN,DAILY,"['12-2015', '01-2016', '02-2016', '03-2016', '...","[3000.0, 950.0, 700.0, 1015.0, 1085.0, 490.0, ...",1330.0,700.0,700.0,945.0,1050.0,1480.0


In [ ]:
print("---------------- Missing values train ----------------")
print(m_train.isna().sum())
print("---------------- Missing values test ----------------")
print(m_test.isna().sum())

---------------- Missing values train ----------------
AccessoryRate              0
Age                     5181
DaysOnDeposit              0
Deposit                    0
ExpectedTermDate           0
FirstPaymentDate           0
ID                         0
MainApplicantGender        0
Occupation                 0
PaymentMethod              0
RatePerUnit                0
Region                  1446
RegistrationDate           0
SupplierName               0
Term                       0
TotalContractValue         0
Town                       0
UpsellDate             27265
rateTypeEntity             0
TransactionDates           0
PaymentsHistory            0
m1                         0
m2                         0
m3                         0
m4                         0
m5                         0
m6                         0
dtype: int64
---------------- Missing values test ----------------
AccessoryRate             0
Age                    1758
DaysOnDeposit             0
Deposit    

In [ ]:
# Check for duplicates
m_train.duplicated().any(), m_test.duplicated().any()

(False, False)

In [ ]:
cat_cols = ['rateTypeEntity','Town','Region','MainApplicantGender','Occupation']
semi_cat_cols = ['AccessoryRate','DaysOnDeposit','Deposit','RatePerUnit']
weak_cat = ['Term','TotalContractValue']
cont_cols = ['Age']
date_cols = ['ExpectedTermDate','FirstPaymentDate','RegistrationDate','UpsellDate']
one_value_cols = ['PaymentMethod','SupplierName']
history_cols = ['TransactionDates','PaymentsHistory']
target_cols = ['m1','m2','m3','m4','m5','m6']

## History columns

In [ ]:
max_len = 52
padded_payments_tot = []
for r in m_train.copy().iterrows():
    r = r[1]
    padding_len = abs(max_len - len(ast.literal_eval(r.PaymentsHistory)))
    padded_payments_tot.append((ast.literal_eval(r.PaymentsHistory) +r[-6:].values.tolist()+ padding_len*[0.])[1:])
m_train["PaymentsHistoryTot"] = padded_payments_tot

padded_payments_tot = []
for r in m_test.copy().iterrows():
    r = r[1]
    padding_len = abs(max_len - len(ast.literal_eval(r.PaymentsHistory)))
    padded_payments_tot.append((ast.literal_eval(r.PaymentsHistory) +r[-6:].values.tolist()+ padding_len*[0.])[1:])
m_test["PaymentsHistoryTot"] = padded_payments_tot

In [ ]:
#taking care of history_cols
max_len = 52
padded_payments = []
padded_TransactionDates = []
pad_lens=[]
for r in m_train.copy().iterrows():
    r = r[1]
    padding_len = abs(max_len - len(ast.literal_eval(r.PaymentsHistory)))
    padded_payments.append(ast.literal_eval(r.PaymentsHistory) + padding_len*[0.])
    padded_TransactionDates.append(ast.literal_eval(r.TransactionDates) + padding_len*[r.RegistrationDate])
    pad_lens.append(padding_len)
m_train["PaymentsHistory"] = padded_payments
m_train["TransactionDates"] = padded_TransactionDates

padded_payments = []
padded_TransactionDates = []
for r in m_test.copy().iterrows():
    r = r[1]
    padding_len = abs(max_len - len(ast.literal_eval(r.PaymentsHistory)))
    padded_payments.append(ast.literal_eval(r.PaymentsHistory) + padding_len*[0.])
    padded_TransactionDates.append(ast.literal_eval(r.TransactionDates) + padding_len*[r.RegistrationDate])
m_test["PaymentsHistory"] = padded_payments
m_test["TransactionDates"] = padded_TransactionDates

In [ ]:
Payment_df_train = pd.DataFrame(np.array(m_train.PaymentsHistory.values.tolist()).reshape((m_train.PaymentsHistory.values.shape[0], max_len)),columns=["P{}".format(i) for i in range(0,max_len) ])
Payment_df_train['ID']=m_train.ID

Payment_df_test = pd.DataFrame(np.array(m_test.PaymentsHistory.values.tolist()).reshape((m_test.PaymentsHistory.values.shape[0], max_len)),columns=["P{}".format(i) for i in range(0,max_len) ])
Payment_df_test['ID']=m_test.ID

Payment_df_train.head()

,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47,P48,P49,P50,P51,ID
0,3050.0,1050.0,910.0,1050.0,1050.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ID_K00S4N4
1,4245.0,980.0,735.0,735.0,1470.0,735.0,980.0,915.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ID_102CV85
2,2245.0,980.0,980.0,1225.0,980.0,980.0,980.0,1225.0,735.0,490.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ID_HXBJFHB
3,2750.0,1000.0,750.0,1000.0,955.0,880.0,280.0,665.0,770.0,420.0,525.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ID_3K9VZ5J
4,3010.0,665.0,1035.0,985.0,1020.0,1120.0,945.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ID_6YU5YRB


In [38]:
def parse_dates(date):
    try:
        return pd.to_datetime(date, format="%Y-%m-%d %H:%M:%S", errors='raise')
    except ValueError:
        return pd.to_datetime('01-' + date, format='%d-%m-%Y')

Transaction_df_train = pd.DataFrame(np.array(m_train.TransactionDates.values.tolist()).reshape((m_train.TransactionDates.values.shape[0], max_len)),columns=["T{}".format(i) for i in range(0,max_len) ])
for i in range(0,max_len):
    Transaction_df_train["T{}".format(i)] = ((Transaction_df_train["T{}".format(i)]).apply(parse_dates)
                                       - (m_train["RegistrationDate"]).apply(parse_dates)).dt.days
Transaction_df_train['ID']=m_train.ID

Transaction_df_test = pd.DataFrame(np.array(m_test.TransactionDates.values.tolist()).reshape((m_test.TransactionDates.values.shape[0], max_len)),columns=["T{}".format(i) for i in range(0,max_len) ])
for i in range(0,max_len):
    Transaction_df_test["T{}".format(i)] = ((Transaction_df_test["T{}".format(i)]).apply(parse_dates)
                                       - (m_test["RegistrationDate"]).apply(parse_dates)).dt.days
Transaction_df_test['ID']=m_test.ID

Transaction_df_train.head()

,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T22,T23,T24,T25,T26,T27,T28,T29,T30,T31,T32,T33,T34,T35,T36,T37,T38,T39,T40,T41,T42,T43,T44,T45,T46,T47,T48,T49,T50,T51,ID
0,-9,22,53,82,113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ID_K00S4N4
1,-17,14,45,74,105,135,166,196,227,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ID_102CV85
2,-24,6,37,68,97,128,158,189,219,250,311,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ID_HXBJFHB
3,-1,30,61,90,121,151,182,212,243,274,304,335,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ID_3K9VZ5J
4,-9,22,53,82,113,143,174,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ID_6YU5YRB


## Date columns

In [39]:
#date
m_train["FirstPaymentYear"] = pd.to_datetime(m_train["FirstPaymentDate"]).dt.year
m_train["FirstPaymentMonth"] = pd.to_datetime(m_train["FirstPaymentDate"]).dt.month

m_test["FirstPaymentYear"] = pd.to_datetime(m_test["FirstPaymentDate"]).dt.year
m_test["FirstPaymentMonth"] = pd.to_datetime(m_test["FirstPaymentDate"]).dt.month
m_train.head()

,AccessoryRate,Age,DaysOnDeposit,Deposit,ExpectedTermDate,FirstPaymentDate,ID,MainApplicantGender,Occupation,PaymentMethod,RatePerUnit,Region,RegistrationDate,SupplierName,Term,TotalContractValue,Town,UpsellDate,rateTypeEntity,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,PaymentsHistoryTot,FirstPaymentYear,FirstPaymentMonth
0,0.0,41.0,7,2000,2016-12-08 00:00:00,2015-12-10 09:52:35,ID_K00S4N4,Male,Other,FINANCED,35,Mount Kenya Region,2015-12-10 00:00:00,d_light,364,14740.0,Embu,NaN,DAILY,"[12-2015, 01-2016, 02-2016, 03-2016, 04-2016, ...","[3050.0, 1050.0, 910.0, 1050.0, 1050.0, 0.0, 0...",1225.0,1050.0,1190.0,525.0,1750.0,1890.0,"[1050.0, 910.0, 1050.0, 1050.0, 1225.0, 1050.0...",2015,12
1,35.0,48.0,7,2000,2017-01-13 00:00:00,2015-12-18 06:22:34,ID_102CV85,Female,Business,FINANCED,35,Nairobi Region,2015-12-18 00:00:00,d_light,392,29480.0,Makueni,2018-03-29 10:14:58,DAILY,"[12-2015, 01-2016, 02-2016, 03-2016, 04-2016, ...","[4245.0, 980.0, 735.0, 735.0, 1470.0, 735.0, 9...",980.0,980.0,1225.0,980.0,935.0,355.0,"[980.0, 735.0, 735.0, 1470.0, 735.0, 980.0, 91...",2015,12
2,0.0,43.0,7,2000,2016-11-23 00:00:00,2015-11-25 13:25:57,ID_HXBJFHB,Female,Teacher,FINANCED,35,NaN,2015-11-25 00:00:00,d_light,364,14740.0,UNKNOWN,NaN,DAILY,"[11-2015, 12-2015, 01-2016, 02-2016, 03-2016, ...","[2245.0, 980.0, 980.0, 1225.0, 980.0, 980.0, 9...",250.0,1000.0,250.0,500.0,560.0,1150.0,"[980.0, 980.0, 1225.0, 980.0, 980.0, 980.0, 12...",2015,11
3,0.0,56.0,7,2000,2016-11-30 00:00:00,2015-12-05 10:34:32,ID_3K9VZ5J,Female,Other,FINANCED,35,Mount Kenya Region,2015-12-02 00:00:00,d_light,364,14740.0,Kirinyaga,NaN,DAILY,"[12-2015, 01-2016, 02-2016, 03-2016, 04-2016, ...","[2750.0, 1000.0, 750.0, 1000.0, 955.0, 880.0, ...",630.0,805.0,700.0,855.0,245.0,775.0,"[1000.0, 750.0, 1000.0, 955.0, 880.0, 280.0, 6...",2015,12
4,0.0,56.0,7,2000,2016-12-08 00:00:00,2015-12-10 15:15:16,ID_6YU5YRB,Male,Other,FINANCED,35,Mount Kenya Region,2015-12-10 00:00:00,d_light,364,14740.0,Meru,NaN,DAILY,"[12-2015, 01-2016, 02-2016, 03-2016, 04-2016, ...","[3010.0, 665.0, 1035.0, 985.0, 1020.0, 1120.0,...",1050.0,985.0,1085.0,980.0,1770.0,90.0,"[665.0, 1035.0, 985.0, 1020.0, 1120.0, 945.0, ...",2015,12


In [40]:
m_train = pd.merge(left=m_train, right=Transaction_df_train, on="ID")
m_train = pd.merge(left=m_train, right=Payment_df_train, on="ID")
m_test = pd.merge(left=m_test, right=Transaction_df_test, on="ID")
m_test = pd.merge(left=m_test, right=Payment_df_test, on="ID")
m_train.drop(one_value_cols+history_cols+date_cols+["ID","PaymentsHistoryTot"],axis='columns',inplace=True)
m_test.drop(one_value_cols+history_cols+date_cols+["ID","PaymentsHistoryTot"],axis='columns',inplace=True)
m_train.head()

,AccessoryRate,Age,DaysOnDeposit,Deposit,MainApplicantGender,Occupation,RatePerUnit,Region,Term,TotalContractValue,Town,rateTypeEntity,m1,m2,m3,m4,m5,m6,FirstPaymentYear,FirstPaymentMonth,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T22,T23,T24,T25,T26,T27,T28,T29,T30,T31,T32,T33,T34,T35,T36,T37,T38,T39,T40,T41,T42,T43,T44,T45,T46,T47,T48,T49,T50,T51,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47,P48,P49,P50,P51
0,0.0,41.0,7,2000,Male,Other,35,Mount Kenya Region,364,14740.0,Embu,DAILY,1225.0,1050.0,1190.0,525.0,1750.0,1890.0,2015,12,-9,22,53,82,113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3050.0,1050.0,910.0,1050.0,1050.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,35.0,48.0,7,2000,Female,Business,35,Nairobi Region,392,29480.0,Makueni,DAILY,980.0,980.0,1225.0,980.0,935.0,355.0,2015,12,-17,14,45,74,105,135,166,196,227,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4245.0,980.0,735.0,735.0,1470.0,735.0,980.0,915.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,43.0,7,2000,Female,Teacher,35,NaN,364,14740.0,UNKNOWN,DAILY,250.0,1000.0,250.0,500.0,560.0,1150.0,2015,11,-24,6,37,68,97,128,158,189,219,250,311,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2245.0,980.0,980.0,1225.0,980.0,980.0,980.0,1225.0,735.0,490.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,56.0,7,2000,Female,Other,35,Mount Kenya Region,364,14740.0,Kirinyaga,DAILY,630.0,805.0,700.0,855.0,245.0,775.0,2015,12,-1,30,61,90,121,151,182,212,243,274,304,335,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2750.0,1000.0,750.0,1000.0,955.0,880.0,280.0,665.0,770.0,420.0,525.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,56.0,7,2000,Male,Other,35,Mount Kenya Region,364,14740.0,Meru,DAILY,1050.0,985.0,1085.0,980.0,1770.0,90.0,2015,12,-9,22,53,82,113,143,174,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3010.0,665.0,1035.0,985.0,1020.0,1120.0,945.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Categorical columns

In [41]:
for col in cat_cols:
    m_train[col] = m_train[col].astype('category').cat.codes
    m_test[col] = m_test[col].astype('category').cat.codes

In [42]:
m_train.head()

,AccessoryRate,Age,DaysOnDeposit,Deposit,MainApplicantGender,Occupation,RatePerUnit,Region,Term,TotalContractValue,Town,rateTypeEntity,m1,m2,m3,m4,m5,m6,FirstPaymentYear,FirstPaymentMonth,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T22,T23,T24,T25,T26,T27,T28,T29,T30,T31,T32,T33,T34,T35,T36,T37,T38,T39,T40,T41,T42,T43,T44,T45,T46,T47,T48,T49,T50,T51,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47,P48,P49,P50,P51
0,0.0,41.0,7,2000,1,5,35,1,364,14740.0,5,0,1225.0,1050.0,1190.0,525.0,1750.0,1890.0,2015,12,-9,22,53,82,113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3050.0,1050.0,910.0,1050.0,1050.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,35.0,48.0,7,2000,0,0,35,2,392,29480.0,22,0,980.0,980.0,1225.0,980.0,935.0,355.0,2015,12,-17,14,45,74,105,135,166,196,227,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4245.0,980.0,735.0,735.0,1470.0,735.0,980.0,915.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,43.0,7,2000,0,6,35,-1,364,14740.0,43,0,250.0,1000.0,250.0,500.0,560.0,1150.0,2015,11,-24,6,37,68,97,128,158,189,219,250,311,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2245.0,980.0,980.0,1225.0,980.0,980.0,980.0,1225.0,735.0,490.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,56.0,7,2000,0,5,35,1,364,14740.0,14,0,630.0,805.0,700.0,855.0,245.0,775.0,2015,12,-1,30,61,90,121,151,182,212,243,274,304,335,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2750.0,1000.0,750.0,1000.0,955.0,880.0,280.0,665.0,770.0,420.0,525.0,735.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,56.0,7,2000,1,5,35,1,364,14740.0,25,0,1050.0,985.0,1085.0,980.0,1770.0,90.0,2015,12,-9,22,53,82,113,143,174,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3010.0,665.0,1035.0,985.0,1020.0,1120.0,945.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Training

In [43]:
#prepare data
test_cols = ["m{}".format(i) for i in range(1,7)]
train_cols = m_train.columns.difference(test_cols)
X = m_train[train_cols]
y =  m_train[test_cols]
X = X.fillna(X.mean())
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

# Instantiate model
l = LinearRegression()
# Fit Model
l.fit(X_train, y_train)
# Predict
y_pred = l.predict(X_test)
print(np.sqrt(mean_squared_error(y_pred,y_test)))
print(np.sqrt(mean_squared_error(l.predict(X_train),y_train)))

714.8910938325777
833.0843339535438


In [44]:
XB = XGBRegressor(n_estimators = 100,learning_rate=0.01,subsample=0.1,random_state=0)
XB = MultiOutputRegressor(XB)
XB.fit(X_train, y_train)
y_pred = XB.predict(X_test)
print(np.sqrt(mean_squared_error(y_pred,y_test)))

719.4594542085051


In [45]:
RF = RandomForestRegressor(n_estimators=620,random_state=42,max_depth=8,max_samples=290)
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
print(np.sqrt(mean_squared_error(y_pred,y_test)))

706.0184299261417


In [46]:
cb = CatBoostRegressor(iterations = 1000,
                        eval_metric='RMSE',
                        loss_function= 'RMSE',
                        #learning_rate=0.9,
                        #use_best_model =True,verbose=100,
                        random_seed=42,
                        #l2_leaf_reg=10000,   #---less overfit (more better val score)
                          )
cb = MultiOutputRegressor(cb)
cb.fit(X_train, y_train)
y_pred = cb.predict(X_test)
print(np.sqrt(mean_squared_error(y_pred,y_test)))

Streaming output truncated to the last 5000 lines.
5:	learn: 837.7444286	total: 159ms	remaining: 26.4s
6:	learn: 828.0933871	total: 186ms	remaining: 26.4s
7:	learn: 816.4573945	total: 219ms	remaining: 27.1s
8:	learn: 807.2485349	total: 244ms	remaining: 26.8s
9:	learn: 798.0897497	total: 268ms	remaining: 26.6s
10:	learn: 789.2254416	total: 294ms	remaining: 26.4s
11:	learn: 780.0444781	total: 320ms	remaining: 26.3s
12:	learn: 771.7475139	total: 344ms	remaining: 26.1s
13:	learn: 765.1356765	total: 367ms	remaining: 25.8s
14:	learn: 757.6070711	total: 392ms	remaining: 25.7s
15:	learn: 749.0376639	total: 419ms	remaining: 25.8s
16:	learn: 741.8049040	total: 446ms	remaining: 25.8s
17:	learn: 733.8226198	total: 471ms	remaining: 25.7s
18:	learn: 726.7533968	total: 496ms	remaining: 25.6s
19:	learn: 721.0509226	total: 519ms	remaining: 25.4s
20:	learn: 715.5494199	total: 543ms	remaining: 25.3s
21:	learn: 710.1483664	total: 567ms	remaining: 25.2s
22:	learn: 704.9366648	total: 591ms	remaining: 25.1s


In [47]:
LB = LGBMRegressor(n_estimators = 200,
                        #learning_rate=0.01,
                        verbose=100,
                        random_seed=42,
                        #l2_leaf_reg=10000,   #---less overfit (more better val score)
                          )
LB = MultiOutputRegressor(LB)
LB.fit(X_train, y_train)
y_pred = LB.predict(X_test)
print(np.sqrt(mean_squared_error(y_pred,y_test)))

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.855740
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.545993
[LightGBM] [Debug] init for col-wise cost 0.010468 seconds, init for row-wise cost 0.021922 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014493 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 16661
[LightGBM] [Info] Number of data points in the train set: 19604, number of used features: 96
[LightGBM] [Info] Start training from score 814.754948
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a

## Submission

In [48]:
x = m_test[train_cols].fillna(m_train.mean())
preds = cb.predict(x)
preds = preds.flatten()
ss["Target"] = preds
ss.to_csv('cb.csv', index=False)
ss.head(n=12)

,ID,Target
0,ID_6L67PAA x m1,922.364447
1,ID_6L67PAA x m2,1115.544673
2,ID_6L67PAA x m3,1021.360994
3,ID_6L67PAA x m4,1000.054774
4,ID_6L67PAA x m5,1155.248160
5,ID_6L67PAA x m6,1206.849538
6,ID_VJ80SX2 x m1,1068.180893
7,ID_VJ80SX2 x m2,994.226281
8,ID_VJ80SX2 x m3,964.479028
9,ID_VJ80SX2 x m4,1050.989285
